In [ ]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('2012_2020_18-fixed-3.csv', index_col=0).sort_index()

In [ ]:
df

In [ ]:
vikane = df.loc[df['localityNo'] == 13704]   

In [ ]:
vikane

In [ ]:
vikane = 13704

#localityNos = [ vikane ]
#[ 13704, 11756, 12982 ]

#localityNos = np.unique(df['localityNo'].to_numpy())[1:100]

localityNos = [11756, 11754, 11667, 11652, 10080, 14435, 13965, 11738, 10086, 39577, 10309, 13704]

# Calculate growing periods

periods = []

for localityNo in localityNos:
    vikane = df.loc[df['localityNo'] == localityNo]    

    lastYearWeek = { 'year': None, 'week': None, 'yearweek': None }
    currentPeriod = { 'localityNo': localityNo, 'startYear': None, 'startWeek': None, 'endYear': None, 'endWeek': None }

    counter = 0

    for index, row in vikane.iterrows():
        counter = counter + 1
        currentYearWeek = row['year'] * 52 + row['week']

        if not lastYearWeek['yearweek']:
            currentPeriod['startYear'] = row['year']
            currentPeriod['startWeek'] = row['week']
        elif lastYearWeek['yearweek'] and not (lastYearWeek['yearweek'] == currentYearWeek - 1 or lastYearWeek['yearweek'] == currentYearWeek - 2):
            currentPeriod['endYear'] = lastYearWeek['year']
            currentPeriod['endWeek'] = lastYearWeek['week']

            periods.append(copy.copy(currentPeriod))

            currentPeriod['startYear'] = row['year']
            currentPeriod['startWeek'] = row['week']
            currentPeriod['endYear'] = None
            currentPeriod['endWeek'] = None
        elif counter == len(vikane):
            currentPeriod['endYear'] = row['year']
            currentPeriod['endWeek'] = row['week']

            periods.append(copy.copy(currentPeriod))

        lastYearWeek['year'] = row['year']
        lastYearWeek['week'] = row['week']
        lastYearWeek['yearweek'] = currentYearWeek

for period in periods:
    print(period)

In [ ]:
allTreatmentInfo = []

#periods = [ {'localityNo': 10090, 'startYear': 2016, 'startWeek': 1, 'endYear': 2016, 'endWeek': 35} ]

for period in periods:
    localityNo = period['localityNo']
    startYearWeek = period['startYear'] * 52 + period['startWeek']
    endYearWeek = period['endYear'] * 52 + period['endWeek']

    treatmentInfo = []
    
    lastTreatmentYearWeek = None
    lastRow = None
    row = None
    lastLevelsBeforeTreatment = None
    lastLevelsAfterTreatment = None
    
    vikane = df.loc[df['localityNo'] == localityNo]
    
    counter = 0

    for index, nextRow in vikane.iterrows():
        counter = counter + 1
        
        if lastRow is not None:
            currentYearWeek = row['year'] * 52 + row['week']
            
            if currentYearWeek < startYearWeek or currentYearWeek > endYearWeek:
                lastRow = row
                row = nextRow
            
                continue
                
            if row['hasMechanicalRemoval'] or row['hasBathTreatment'] or (counter == len(vikane) and (nextRow['hasMechanicalRemoval'] or nextRow['hasBathTreatment'])):
                beforeRow = lastRow if lastRow['avgAdultFemaleLice'] > row['avgAdultFemaleLice'] else row
                afterRow = nextRow if lastRow['avgAdultFemaleLice'] < row['avgAdultFemaleLice'] else row

                currentLevelsBeforeTreatment = { 'avgAdultFemaleLice': beforeRow['avgAdultFemaleLice'], 'avgMobileLice': beforeRow['avgMobileLice'], 'avgStationaryLice': beforeRow['avgStationaryLice'] }
                currentLevelsAfterTreatment = { 'avgAdultFemaleLice': afterRow['avgAdultFemaleLice'], 'avgMobileLice': afterRow['avgMobileLice'], 'avgStationaryLice': afterRow['avgStationaryLice'] }

                treatmentYearWeek = afterRow['year'] * 52 + afterRow['week']

                if lastTreatmentYearWeek:    
                    weeks = treatmentYearWeek - lastTreatmentYearWeek
                    treatmentInfo.append(copy.copy({ 'hasMechanicalRemoval': row['hasMechanicalRemoval'] or nextRow['hasMechanicalRemoval'], 'hasBathTreatment': row['hasBathTreatment'] or nextRow['hasBathTreatment'], 'lastLevelsBeforeTreatment': lastLevelsBeforeTreatment, 'lastLevelsAfterTreatment': lastLevelsAfterTreatment, 'currentLevelsBeforeTreatment': currentLevelsBeforeTreatment, 'currentLevelsAfterTreatment': currentLevelsAfterTreatment, 'weeks': weeks }))

                lastTreatmentYearWeek = treatmentYearWeek

                lastLevelsBeforeTreatment = currentLevelsBeforeTreatment
                lastLevelsAfterTreatment = currentLevelsAfterTreatment

            
            lastRow = row
            row = nextRow
        elif row is not None:
            lastRow = row 
            row = nextRow
        else:
            row = nextRow
                
    allTreatmentInfo = allTreatmentInfo + treatmentInfo
    
print(len(allTreatmentInfo))

In [ ]:
#allTreatmentInfo[0]

In [ ]:
def getTreatmentMetrics(currentTreatmentInfo):
    beforeTreatmentAdultFemale = np.array([ info['lastLevelsBeforeTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
    beforeTreatmentMobile = np.array([ info['lastLevelsBeforeTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
    beforeTreatmentStationary = np.array([ info['lastLevelsBeforeTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

    afterTreatmentAdultFemale = np.array([ info['lastLevelsAfterTreatment']['avgAdultFemaleLice'] for info in currentTreatmentInfo ])
    afterTreatmentMobile = np.array([ info['lastLevelsAfterTreatment']['avgMobileLice'] for info in currentTreatmentInfo ])
    afterTreatmentStationary = np.array([ info['lastLevelsAfterTreatment']['avgStationaryLice'] for info in currentTreatmentInfo ])

    allTreatmentWeeks = np.array([ info['weeks'] for info in currentTreatmentInfo ])
    allMechanicalTreatments = np.array([ info['hasMechanicalRemoval'] for info in currentTreatmentInfo ])
    allBathTreatments = np.array([ info['hasBathTreatment'] for info in currentTreatmentInfo ])

    treatmentEffectAdultFemale = beforeTreatmentAdultFemale - afterTreatmentAdultFemale
    treatmentEffectMobile = beforeTreatmentMobile - afterTreatmentMobile
    treatmentEffectStationary = beforeTreatmentStationary - afterTreatmentStationary

    colors = ['r' if t else 'g' for t in allMechanicalTreatments]
    
    return {
        'treatmentInfo': currentTreatmentInfo,
        
        'beforeTreatmentAdultFemale': beforeTreatmentAdultFemale,
        'beforeTreatmentMobile': beforeTreatmentMobile,
        'beforeTreatmentStationary': beforeTreatmentStationary,
        
        'afterTreatmentAdultFemale': afterTreatmentAdultFemale,
        'afterTreatmentMobile': afterTreatmentMobile,
        'afterTreatmentStationary': afterTreatmentStationary,
        
        'allTreatmentWeeks': allTreatmentWeeks,
        'allMechanicalTreatments': allMechanicalTreatments,
        'allBathTreatments': allBathTreatments,
        
        'treatmentEffectAdultFemale': treatmentEffectAdultFemale,
        'treatmentEffectMobile': treatmentEffectMobile,
        'treatmentEffectStationary': treatmentEffectStationary,
        
        'colors': colors
    }

In [ ]:
mechanicalTreatmentInfos = [ info for info in allTreatmentInfo if info['hasMechanicalRemoval'] ]
bathTreatmentInfos = [ info for info in allTreatmentInfo if info['hasBathTreatment'] ]

allMetrics = getTreatmentMetrics(allTreatmentInfo)
mechanicalMetrics = getTreatmentMetrics(mechanicalTreatmentInfos)
bathMetrics = getTreatmentMetrics(bathTreatmentInfos)

#plt.scatter(allMetrics['treatmentEffectAdultFemale'], allMetrics['allTreatmentWeeks'], c=colors)

In [ ]:
def calculateGrowthWeeks(liceLevels, allTreatmentWeeks, subset):
    x = liceLevels[subset]
    Y = allTreatmentWeeks[subset]

    X = x.reshape(-1, 1)

    #plt.plot(X, Y, 'p')

    reg = LinearRegression().fit(X, Y)

    #print(reg.intercept_, reg.coef_)
    #print(reg.score(X, Y))
    
    return (reg.coef_ * -1, reg.score(X, Y))

In [ ]:
def plotMetrics(metrics):
    subsetTEAdultFemale = np.where((metrics['treatmentEffectAdultFemale'] < 2) & (metrics['treatmentEffectAdultFemale'] >= 0))
    subsetTEMobile = np.where((metrics['treatmentEffectMobile'] < 7) & (metrics['treatmentEffectMobile'] >= 0))
    subsetTEStationary = np.where((metrics['treatmentEffectStationary'] < 5) & (metrics['treatmentEffectStationary'] >= 0))

    (treatmentEffectWeeksAdultFemale, treatmentEffectRSQAdultFemale) = calculateGrowthWeeks(metrics['treatmentEffectAdultFemale'], metrics['allTreatmentWeeks'], subsetTEAdultFemale)
    (treatmentEffectWeeksMobile, treatmentEffectRSQMobile) = calculateGrowthWeeks(metrics['treatmentEffectMobile'], metrics['allTreatmentWeeks'], subsetTEMobile)
    (treatmentEffectWeeksStationary, treatmentEffectRSQStationary) = calculateGrowthWeeks(metrics['treatmentEffectStationary'], metrics['allTreatmentWeeks'], subsetTEStationary)
    
    subsetBTAdultFemale = np.where((metrics['beforeTreatmentAdultFemale'] < 1))
    subsetBTMobile = np.where((metrics['beforeTreatmentMobile'] < 10))
    subsetBTStationary = np.where((metrics['beforeTreatmentStationary'] < 5))

    (beforeTreatmentWeeksAdultFemale, beforeTreatmentRSQAdultFemale) = calculateGrowthWeeks(metrics['beforeTreatmentAdultFemale'], metrics['allTreatmentWeeks'], subsetBTAdultFemale)
    (beforeTreatmentWeeksMobile, beforeTreatmentRSQMobile) = calculateGrowthWeeks(metrics['beforeTreatmentMobile'], metrics['allTreatmentWeeks'], subsetBTMobile)
    (beforeTreatmentWeeksStationary, beforeTreatmentRSQStationary) = calculateGrowthWeeks(metrics['beforeTreatmentStationary'], metrics['allTreatmentWeeks'], subsetBTStationary)
   
    subsetATAdultFemale = np.where((metrics['afterTreatmentAdultFemale'] < 0.5))
    subsetATMobile = np.where((metrics['afterTreatmentMobile'] < 1))
    subsetATStationary = np.where((metrics['afterTreatmentStationary'] < 1))

    (afterTreatmentWeeksAdultFemale, afterTreatmentRSQAdultFemale) = calculateGrowthWeeks(metrics['afterTreatmentAdultFemale'], metrics['allTreatmentWeeks'], subsetATAdultFemale)
    (afterTreatmentWeeksMobile, afterTreatmentRSQMobile) = calculateGrowthWeeks(metrics['afterTreatmentMobile'], metrics['allTreatmentWeeks'], subsetATMobile)
    (afterTreatmentWeeksStationary, afterTreatmentRSQStationary) = calculateGrowthWeeks(metrics['afterTreatmentStationary'], metrics['allTreatmentWeeks'], subsetATStationary)
    
    metricsText = [
        '',
        'Weeks between treatment: %0.2f (%i days)' % (np.mean(metrics['allTreatmentWeeks']), np.mean(metrics['allTreatmentWeeks']) * 7),
        
        '',
        'Treatment Effectiveness',
        '              Before   After   Pct Removed',
        'Adult Female  %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentAdultFemale']), np.mean(metrics['afterTreatmentAdultFemale']), (1 - np.mean(metrics['afterTreatmentAdultFemale']) / np.mean(metrics['beforeTreatmentAdultFemale'])) * 100),   
        'Mobile        %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentMobile']), np.mean(metrics['afterTreatmentMobile']), (1 - np.mean(metrics['afterTreatmentMobile']) / np.mean(metrics['beforeTreatmentMobile'])) * 100),
        'Stationary    %0.2f     %0.2f    %i%%' % (np.mean(metrics['beforeTreatmentStationary']), np.mean(metrics['afterTreatmentStationary']), (1 - np.mean(metrics['afterTreatmentStationary']) / np.mean(metrics['beforeTreatmentStationary'])) * 100),
        
        '',
        #'Additional weeks between treatment per removed lice',
        #'AF: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksAdultFemale, treatmentEffectRSQAdultFemale),
        #'M: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksMobile, treatmentEffectRSQMobile),
        #'S: %0.2f (R^2 = %0.2f)' % (treatmentEffectWeeksStationary, treatmentEffectRSQStationary),
        #'Additional weeks between treatment per lice (before)',
        #'AF: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksAdultFemale, beforeTreatmentRSQAdultFemale),
        #'M: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksMobile, beforeTreatmentRSQMobile),
        #'S: %0.2f (R^2 = %0.2f)' % (beforeTreatmentWeeksStationary, beforeTreatmentRSQStationary),
        
        '',
        'Additional weeks between treatment per lice (after)',
        'Adult Female  %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksAdultFemale, afterTreatmentRSQAdultFemale),
        'Mobile        %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksMobile, afterTreatmentRSQMobile),
        'Stationary    %0.2f (R^2 = %0.2f)' % (afterTreatmentWeeksStationary, afterTreatmentRSQStationary)
    ]

    for metric in metricsText:
        print(metric)

print('FARM: Blom Fiskeoppdrett')
print('SITE: All sites')

print()
print('START DATE: 2012 Week 1')
print('END DATE: 2020 Week 18')
print()

print('----------')


print()
print('ALL TREATMENTS (%i)' % (len(allMetrics['treatmentInfo'], )))
plotMetrics(allMetrics)

print()
print('----------')
print()
print('MECHANICAL TREATMENTS (%i)' % (len(mechanicalMetrics['treatmentInfo'], )))
plotMetrics(mechanicalMetrics)

print()
print('----------')
print()
print('BATH TREATMENTS (%i)' % (len(bathMetrics['treatmentInfo'], )))
plotMetrics(bathMetrics)

In [ ]:
# subset = allMetrics['afterTreatmentAdultFemale'] < 1

# plt.hist(allMetrics['afterTreatmentAdultFemale'][subset], bins=20)

In [ ]:
# subset = allMetrics['afterTreatmentMobile'] < 1

# plt.hist(allMetrics['afterTreatmentMobile'][subset], bins=20)

In [ ]:
# subset = allMetrics['afterTreatmentStationary'] < 1

# plt.hist(allMetrics['afterTreatmentStationary'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentAdultFemale'] < 1

# plt.hist(allMetrics['beforeTreatmentAdultFemale'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentMobile'] < 10

# plt.hist(allMetrics['beforeTreatmentMobile'][subset], bins=20)

In [ ]:
# subset = allMetrics['beforeTreatmentStationary'] < 5

# plt.hist(allMetrics['beforeTreatmentStationary'][subset], bins=20)

In [ ]:
# subset = np.where((treatmentEffectAdultFemale < 0.5) &  (treatmentEffectAdultFemale > -0.25))

# x = treatmentEffectAdultFemale[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentStationary < 0.5))

# x = afterTreatmentStationary[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentMobile < 0.5))

# x = afterTreatmentMobile[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')

In [ ]:
# subset = np.where((afterTreatmentAdultFemale < 0.5))

# x = afterTreatmentAdultFemale[subset]
# Y = allTreatmentWeeks[subset]

# X = x.reshape(-1, 1)

# plt.plot(X, Y, 'p')

# reg = LinearRegression().fit(X, Y)

# print(reg.intercept_, reg.coef_)
# print(reg.score(X, Y))

# predTreatmentWeeks = reg.predict(X)

# plt.plot(X, predTreatmentWeeks, '-')